In [1]:
# Get data from CSV
import csv
data = []
with open('data.csv') as csvfile:
    reader = csv.DictReader(csvfile)    
    for row in reader:
        text = " ".join(row.values())
        data.append(text)
        
data[0]

'No es necesaria. Analista Junior de Finanzas y Gobierno Corporativo •\tRecopilación, procesamiento y análisis de información cualitativa y cuantitativa: La recopilación de información se aborda a través de la interacción con clientes, la revisión de documentación interna y la búsqueda de información pública. Posteriormente la misma es procesada en conjunto con los demás consultores con el objetivo de generar conclusiones que se traduzcan en acciones específicas.\n•\tAsistencia en el desarrollo de publicaciones: Periódicamente se realizan estudios relacionados a mercados financieros y de capitales que son publicados en diarios y revistas de reconocido prestigio. \n•\tAsistencia en el desarrollo de proyectos: En su mayoría, los mismos están dirigidos a identificar ámbitos de mejora en materia de mejores prácticas de Gobierno Corporativo, y eventualmente guiar su implementación. Ello aplica tanto para entidades públicas y privadas.'

In [2]:
# Preprocessing the data

punctuations = ['•','/']
translator = str.maketrans("".join(punctuations),' '*len(punctuations))

proc_data = []
for text in data:
    text = text.lower()
    text = text.translate(translator)
    proc_data.append(text)
    
data = proc_data
data[0]

'no es necesaria. analista junior de finanzas y gobierno corporativo  \trecopilación, procesamiento y análisis de información cualitativa y cuantitativa: la recopilación de información se aborda a través de la interacción con clientes, la revisión de documentación interna y la búsqueda de información pública. posteriormente la misma es procesada en conjunto con los demás consultores con el objetivo de generar conclusiones que se traduzcan en acciones específicas.\n \tasistencia en el desarrollo de publicaciones: periódicamente se realizan estudios relacionados a mercados financieros y de capitales que son publicados en diarios y revistas de reconocido prestigio. \n \tasistencia en el desarrollo de proyectos: en su mayoría, los mismos están dirigidos a identificar ámbitos de mejora en materia de mejores prácticas de gobierno corporativo, y eventualmente guiar su implementación. ello aplica tanto para entidades públicas y privadas.'

In [3]:
from nltk.tokenize import word_tokenize
from nltk.tag.stanford import StanfordPOSTagger

spanish_postagger = StanfordPOSTagger('models/spanish.tagger', './stanford-postagger.jar')

terms_by_tag = {}
terms_by_tag['n'] = set()
terms_by_tag['v'] = set()
terms_by_tag['a'] = set()
terms_by_tag['c'] = set()
terms_by_tag['d'] = set()
terms_by_tag['f'] = set()
terms_by_tag['i'] = set()
terms_by_tag['p'] = set()
terms_by_tag['r'] = set()
terms_by_tag['s'] = set()
terms_by_tag['w'] = set()
terms_by_tag['z'] = set()


for text in data:
    tag_terms = spanish_postagger.tag(word_tokenize(text))
    for term in tag_terms:
        if term[0] == 'kardex':
            print(term[1])
        for tag in terms_by_tag:
            if term[1][0] == tag:
                terms_by_tag[tag].add(term[0])     

/home/howl/.virtualenvs/TextClasification/lib/python3.5/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


z0
Loading default properties from tagger models/spanish.tagger
Loading POS tagger from models/spanish.tagger ... done [0.4 sec].



OSError: Java command failed : ['/usr/bin/java', '-mx1000m', '-cp', './stanford-postagger.jar', 'edu.stanford.nlp.tagger.maxent.MaxentTagger', '-model', 'models/spanish.tagger', '-textFile', '/tmp/tmp7kcpjzju', '-tokenize', 'false', '-outputFormatOptions', 'keepEmptySentences', '-encoding', 'utf8']